# Project1 - Team 5, Studio 1

Annie Chu, SeungU Lyu

### Question: What would the age demographics be in 15 years if the US implemented an one-child policy?

In [16]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

# importing "copy" for copy operations 
import copy

### Initial Data Pulls

#### Initial Population Data by Age Range and Gender (0-4, 5-9, etc)

In [17]:
pop_init_data = pd.read_csv('data/pop_2017_US.csv', index_col='age')

,male,female
age,,
4,10195968,9742892
9,10368141,9936097
14,10605072,10173382
19,10800491,10331169
24,11349142,10769493
29,11902230,11468230
34,11089131,10883081
39,10615985,10616012
44,9753115,9890258


Source: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk

#### Death Rate Data by Age Group and Gender (0, 1-4, 5-9, etc)

In [18]:
death_rate_data = pd.read_csv('data/age_death_rate.csv', index_col='age')

,male,female
age,,
0,0.006392,0.005377
4,0.000280,0.000216
9,0.000132,0.000102
14,0.000169,0.000122
19,0.000666,0.000291
24,0.001299,0.000465
29,0.001505,0.000608
34,0.001709,0.000835
39,0.001985,0.001101


Source: https://www.statista.com/statistics/241488/population-of-the-us-by-sex-and-age/

#### Probability of Woman Ever Having a Child by Age Group (15-19, 20-24, etc)

In [19]:
child_ever_data = pd.read_csv('data/child_ever_born.csv', index_col='age')

,percentage
age,
19,0.051
24,0.286
29,0.506
34,0.718
39,0.828
44,0.849
49,0.832


Source: Fertility of Women in the United States: 2012, Lindsay M. Monte and Renee R. Ellis

#### Probability of Woman Having First Child by Age

In [20]:
first_birth_data = pd.read_csv('data/first_birth_rate.csv', index_col='age')

,percentage
age,
15,0.0090
16,0.0182
17,0.0314
18,0.0470
19,0.0597
20,0.0661
21,0.0664
22,0.0650
23,0.0655


Source: https://www.cdc.gov/nchs/nvss/births.htm

#### Adapting Initial Population Data into Age Groups

In [21]:
male_pop = linspace(0,0,90)
female_pop = linspace(0,0,90)

ident = 4
for i in range(90):
    if i>ident:
        ident += 5
    male_pop[i] = int(pop_init_data.male[ident]/5)
    female_pop[i] = int(pop_init_data.female[ident]/5)

In [22]:
female_pop[0] #test

1948578.0

#### Setting Initial Population State by Gender

In [23]:
init_population = State(male = male_pop, female = female_pop)

,values
male,"[2039193.0, 2039193.0, 2039193.0, 2039193.0, 2..."
female,"[1948578.0, 1948578.0, 1948578.0, 1948578.0, 1..."


#### Setting Parameters

In [24]:
system = System(male_death = death_rate_data.male,
               female_death = death_rate_data.female,
               child_ever = child_ever_data.percentage,
               first_rate = first_birth_data.percentage,
               t_0 = 2018,
               t_end = 2033,
               init_pop = init_population)

,values
male_death,age 0 0.006392 4 0.000280 9 0.0001...
female_death,age 0 0.005377 4 0.000216 9 0.0001...
child_ever,age 19 0.051 24 0.286 29 0.506 34 ...
first_rate,age 15 0.0090 16 0.0182 17 0.0314 18 ...
t_0,2018
t_end,2033
init_pop,"male [2039193.0, 2039193.0, 2039193.0, 20..."


#### Splitting Initial Female Population --> Female w/ no children, Female w/ children

In [25]:
population = State(male = copy.deepcopy(system.init_pop.male), female = copy.deepcopy(system.init_pop.female), female_w = copy.deepcopy(system.init_pop.female))

,values
male,"[2039193.0, 2039193.0, 2039193.0, 2039193.0, 2..."
female,"[1948578.0, 1948578.0, 1948578.0, 1948578.0, 1..."
female_w,"[1948578.0, 1948578.0, 1948578.0, 1948578.0, 1..."


#### Assigning Population Value to 2 Female Groups Above

In [26]:
ident = 19
for i in range(15,49):
    if i>ident:
        ident += 5
    population.female[i] = int(population.female[i]*(1-system.child_ever[ident]))
    population.female_w[i] = int(population.female_w[i]*system.child_ever[ident])

In [27]:
#def update_func_male(male_pop,system):

Methods
Parameters: 
1. Initial Population by age and gender group.
2. Death rate by age and gender group.
3. First birth rate for women by age group.
4. Probability of child birth gender (assume 50/50?)
5. Percentage of women with child ever born

State: 
1. Updated population by age and gender group
2. Time


Limitations:

1. Not accounting for twins
2. birth/death rate assuming constant for next 15 years
3. Assumption of gender birth ratio
4. No immigrants
5. Data not the most recent 


Results
1. Compare with US Gov Data for projected future population number 
2. Plot general population vs time (line graph)
3. Plot general populations (with subsections of age groups) vs time (bar graph)


To Do List
1. 3 Update Functions
2. Assume 50/50 gender birth ratio
3. Graph (to test, after each updatefunc?)

